In [1]:
import json
import time

from src.experiments.aux_code import *
from datasets.get_datasets import *
import time
from src.brkga.genetic import genetic as brkga
from src.brkga_variation.genetic import genetic as brkga_var
from src.s_genetic.genetic import *
from src.predicate import *

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
source='workedunder'
target='advisedby'

In [3]:
bk = json.loads(open('src/experiments/kb.txt').readline())


In [4]:
kb_source = bk['imdb']
kb_target = bk['uwcse']

In [5]:
pred_target = create_pred_target(kb_target)

In [6]:
imdb_dataset = datasets.load('imdb', kb_source, target=source, seed=441773, balanced=0) #facts, pos, neg
uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773, balanced=0) #facts, pos, neg

train_facts_source, train_pos_source, train_neg_source = get_train_division(imdb_dataset)
background_knowledge_src = boostsrl.modes(kb_source, [source], useStdLogicVariables=False, 
                                      maxTreeDepth=3, nodeSize=2, numOfClauses=8)
model_src = boostsrl.train(background_knowledge_src, train_pos_source, train_neg_source, train_facts_source, trees=10)

structured_src, new_src_struct = create_structured_trees(model_src)

In [7]:
uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773, balanced=False) #facts, pos, neg
train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset, 0)

0


# Crossover original

## Sem revisão

In [8]:
start = time.time()
res_c_r = genetic(new_src_struct, target, source, train_pos, 
                                         train_neg, train_facts, kb_source, kb_target, pred_target,
                                          NUM_GEN=14, pop_size=30)
print("FIM DO GROOT GENETIC: ", time.time()-start)

/Users/leticia.figueiredo/opt/anaconda3/envs/groot/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/Users/leticia.figueiredo/opt/anaconda3/envs/groot/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


GENERATION:  0
MELHOR RESULTADO:  -0.4718346366070692
BEST: {'m_auc_pr': 0.38296325, 'm_auc_roc': 0.93727275, 'm_cll': -0.27562125000000004, 'm_rec': 0.49810599999999994, 'm_pred': 0.513717375, 'm_f1': 0.38730175, 's_auc_pr': 0.08505931323310514, 's_auc_roc': 0.00465756649845174, 's_cll': 0.017738092504198423, 's_rec': 0.2908104973285868, 's_prec': 0.18377602485970354, 's_f1': 0.08201370935512856}
GENERATION:  1
MELHOR RESULTADO:  -0.4828078975919149
BEST: {'m_auc_pr': 0.38296325, 'm_auc_roc': 0.93727275, 'm_cll': -0.29401125, 'm_rec': 0.49810599999999994, 'm_pred': 0.513717375, 'm_f1': 0.38730175, 's_auc_pr': 0.08505931323310514, 's_auc_roc': 0.00465756649845174, 's_cll': 0.014350701435382862, 's_rec': 0.2908104973285868, 's_prec': 0.18377602485970354, 's_f1': 0.08201370935512856}
GENERATION:  2
MELHOR RESULTADO:  -0.4877334085776906
BEST: {'m_auc_pr': 0.38296325, 'm_auc_roc': 0.93727275, 'm_cll': -0.30203149999999995, 'm_rec': 0.49810599999999994, 'm_pred': 0.513717375, 'm_f1': 0.387

KeyboardInterrupt: 

In [20]:
res_c_r

(<src.s_genetic.population.Population at 0x7fabae3a5668>,
 [-0.38296325,
  -0.38296325,
  -0.38296325,
  -0.38296325,
  -0.38296325,
  -0.38296325,
  -0.38296325,
  -0.38296325,
  -0.38296325],
 [{'m_auc_pr': 0.38296325,
   'm_auc_roc': 0.93727275,
   'm_cll': -0.26704649999999996,
   'm_rec': 0.82777775,
   'm_pred': 0.413420375,
   'm_f1': 0.30366925,
   's_auc_pr': 0.08505931323310514,
   's_auc_roc': 0.00465756649845174,
   's_cll': 0.02578757784767696,
   's_rec': 0.29829768719382904,
   's_prec': 0.2399766913535237,
   's_f1': 0.09002792863710406},
  {'m_auc_pr': 0.38296325,
   'm_auc_roc': 0.93727275,
   'm_cll': -0.27893975,
   'm_rec': 0.82777775,
   'm_pred': 0.413420375,
   'm_f1': 0.30366925,
   's_auc_pr': 0.08505931323310514,
   's_auc_roc': 0.00465756649845174,
   's_cll': 0.04337349708851595,
   's_rec': 0.29829768719382904,
   's_prec': 0.2399766913535237,
   's_f1': 0.09002792863710406},
  {'m_auc_pr': 0.38296325,
   'm_auc_roc': 0.93727275,
   'm_cll': -0.28249725,
 

In [21]:
def get_k_best_individuals(population, k):
    best_ind = []
    all_fitness = []
    for i in range(0, len(population)):
        all_fitness.append((population[i].fitness.values[0], i))
    
    sorted_ind = sorted(all_fitness, reverse=False, key=lambda tup: tup[0])
    for i in range(0, k):
        best_ind.append(population[sorted_ind[i][1]])
    return best_ind

In [22]:
best_cll = res[0].population[0].fitness.values[0]
best_auc = -res[0].population[0].fitness.values[1]
best = res[0].population[0]
for i in res[0].population:
    if i.fitness.values[0] > best_cll and i.fitness.values[1] > best_auc:
        best = i

NameError: name 'res' is not defined

In [23]:
# final_results[f'groot_test:{source}->{target}'] = []
# final_results[f'groot_test_rev:{source}->{target}'] = []
# final_results[f'inf_res:{source}->{target}'] = []
# final_results[f'inf_res_rev:{source}->{target}'] = []

# uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773)
# train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset, 1)
# tr_train_pos = []
# tr_train_neg = []
# tr_train_facts = []

# for i in range(0, len(train_pos)):
#     tr_train_pos.extend(train_pos[i])
#     tr_train_neg.extend(train_neg[i])
#     tr_train_facts.extend(train_facts[i])

# train = [tr_train_pos, tr_train_neg, tr_train_facts]
test = [test_pos, test_neg, test_facts]

individuals = get_k_best_individuals(res_c_r[0].population, 3)

n_ind = 1
for individual in individuals:
    print("INDIVIDUO ", n_ind)
    refine, transfer = get_refine_transfer(individual, source, target, 'imdb', 'uwcse')
    for i in range(len(train_pos)):
        ttrain = [train_pos[i], train_neg[i][:len(train_pos[i])], train_facts[i]]
#         res_ = test_refine_transfer(kb_target, target, refine, transfer, ttrain, test)
        res_ = test_tree_b(source, target, kb_source, kb_target, refine, ttrain, test, transfer)
        print(res_)
#         final_results[f'groot_test:{source}->{target}'].append(res)
#         import os
    #     thisFile = f'boostsrl/best/test/results_{target.replace('recursion_', '')}.db'
    #     thisFile = f'boostsrl/best/test/results_{target}.db'
    #     base = os.path.splitext(thisFile)[0]
    #     os.rename(thisFile, base + ".txt")
    #     tt = open('boostsrl/best/test/results_advisedby.txt', 'r').readlines()
    #     final = []
    #     for i in tt:
    #         final.append(i.replace('\n', ''))
    #     final_results[f'inf_res:{source}->{target}'].append(final)
        print("===========================================================================")
#         break

    n_ind += 1

INDIVIDUO  1
******************************************
Performing Parameter Learning
******************************************
Refine
0;;workedunder00(A,B):- director01(B), actor01(A);true;false
0;true;movie02(C, A), movie03(C, B).;false;false
1;;workedunder10(A,B):- director11(B), actor11(A);true;false
1;true;movie12(C, A), movie13(C, B).;false;false
2;;workedunder20(A,B):- director21(B), actor21(A);true;false
2;true;movie22(C, A), movie23(C, B).;false;false
3;;workedunder30(A,B):- actor31(A), director31(B);true;false
3;true;movie32(C, A), movie33(C, B).;false;false
4;;workedunder40(A,B):- director41(B), actor41(A);true;false
4;true;movie42(C, A), movie43(C, B).;false;false
5;;workedunder50(A,B):- actor51(A), director51(B);true;false
5;true;movie52(C, A), movie53(C, B).;false;false
6;;workedunder60(A,B):- director61(B), actor61(A);true;false
6;true;movie62(C, A), movie63(C, B).;false;false
7;;workedunder70(A,B):- actor71(A), director71(B);true;false
7;true;movie72(C, A), movie73(C, 

Results
   AUC ROC   = 0.937707
   AUC PR    = 0.29769
   CLL	      = -0.584933
   Precision = 0.036881 at threshold = 0.5
   Recall    = 1.0
   F1        = 0.071138


Total learning time: 0.778 seconds
Total inference time: 1.042 seconds
AUC ROC: 0.937707


Results scoring model
   AUC ROC   = 0.95875
   AUC PR    = 0.953338
   CLL	      = -0.302634
   Precision = 0.869565 at threshold = 0.5
   Recall    = 1.0
   F1        = 0.930233


Total scoring time: 0.464 seconds
Parameter learned model CLL: -0.302634


Structure after Parameter Learning
[['advisedby(A, B)', {'': 'student(B), student(A)', 'true': 'publication(C, A), publication(C, B)'}, {'true,true': [0.0, 3, 0], 'true,false': [0.0, 14, 0], 'false': [1.615, 3, 20]}], ['advisedby(A, B)', {'': 'student(B), student(A)', 'true': 'publication(C, A), publication(C, B)'}, {'true,true': [0.0, 3, 0], 'true,false': [5.27e-09, 14, 0], 'false': [1.615, 3, 20]}], ['advisedby(A, B)', {'': 'student(B), student(A)', 'true': 'publication(C, A), 

Results
   AUC ROC   = 0.894739
   AUC PR    = 0.380829
   CLL	      = -0.734249
   Precision = 0.037158 at threshold = 0.5
   Recall    = 0.971429
   F1        = 0.071579


Total learning time: 5.31 seconds
Total inference time: 1.446 seconds
AUC ROC: 0.894739


Results scoring model
   AUC ROC   = 1.0
   AUC PR    = 1.0
   CLL	      = -0.136352
   Precision = 1.0 at threshold = 0.5
   Recall    = 1.0
   F1        = 1.0


Total scoring time: 0.529 seconds
Refined model CLL: -0.136352


Refining iteration 2
********************************
Candidate for revision
0;;advisedby(A, B) :- student(B), student(A).;true;true
0;true;publication(C, A), publication(C, B).;true;true
0;false;ta(D, A, E), publication(F, A).;true;true
1;;advisedby(A, B) :- student(B), student(A).;true;true
1;true;publication(C, A), publication(C, B).;true;true
1;true,false;publication(D, B).;true;true
1;true,false,true;inphase(A, E), inphase(B, E).;true;true
1;false;ta(F, A, G), publication(H, A).;true;true
1;false,f

Results
   AUC ROC   = 0.897299
   AUC PR    = 0.355137
   CLL	      = -0.732869
   Precision = 0.037158 at threshold = 0.5
   Recall    = 0.971429
   F1        = 0.071579


Total learning time: 2.14 seconds
Total inference time: 1.385 seconds
AUC ROC: 0.897299


Results scoring model
   AUC ROC   = 1.0
   AUC PR    = 1.0
   CLL	      = -0.135591
   Precision = 1.0 at threshold = 0.5
   Recall    = 1.0
   F1        = 1.0


Total scoring time: 0.462 seconds
Refined model CLL: -0.135591


Refining iteration 3
********************************
Candidate for revision
0;;advisedby(A, B) :- student(B), student(A).;true;true
0;true;publication(C, A), publication(C, B).;true;true
0;false;ta(D, A, E), publication(F, A).;true;true
1;;advisedby(A, B) :- student(B), student(A).;true;true
1;true;publication(C, A), publication(C, B).;true;true
1;true,false;publication(D, B).;true;true
1;true,false,true;inphase(A, E), inphase(B, E).;true;true
1;false;ta(F, A, G), publication(H, A).;true;true
1;false,f

Results
   AUC ROC   = 0.897299
   AUC PR    = 0.355137
   CLL	      = -0.732869
   Precision = 0.037158 at threshold = 0.5
   Recall    = 0.971429
   F1        = 0.071579


Total learning time: 1.666 seconds
Total inference time: 1.408 seconds
AUC ROC: 0.897299


Results scoring model
   AUC ROC   = 1.0
   AUC PR    = 1.0
   CLL	      = -0.135591
   Precision = 1.0 at threshold = 0.5
   Recall    = 1.0
   F1        = 1.0


Total scoring time: 0.478 seconds
Refined model CLL: -0.135591


******************************************
Best model found
******************************************
Results
   AUC ROC   = 0.897299
   AUC PR    = 0.355137
   CLL	      = -0.732869
   Precision = 0.037158 at threshold = 0.5
   Recall    = 0.971429
   F1        = 0.071579


Total learning time: 2.14 seconds
Total inference time: 1.385 seconds
AUC ROC: 0.897299
Total revision time: 11.827
Best scored revision CLL: -0.135591


[<boostsrl.boostsrl.train object at 0x7fabad755e10>, {'AUC ROC': 0.897299, '

...

Results
   AUC ROC   = 0.864498
   AUC PR    = 0.068086
   CLL	      = -0.602732
   Precision = 0.065606 at threshold = 0.5
   Recall    = 0.942857
   F1        = 0.122677


Total learning time: 4.387 seconds
Total inference time: 1.337 seconds
AUC ROC: 0.864498


Results scoring model
   AUC ROC   = 0.990358
   AUC PR    = 0.991063
   CLL	      = -0.182718
   Precision = 1.0 at threshold = 0.5
   Recall    = 0.878788
   F1        = 0.935484


Total scoring time: 0.473 seconds
Refined model CLL: -0.182718


Refining iteration 2
********************************
Candidate for revision
0;;advisedby(A, B) :- student(B), student(A).;true;true
0;true;publication(C, A), publication(C, B).;true;true
0;false;ta(D, A, E), publication(F, A).;true;true
0;false,true;publication(F, B).;true;true
1;;advisedby(A, B) :- student(B), student(A).;true;true
1;true;publication(C, A), publication(C, B).;true;true
1;false;ta(D, A, E), publication(F, A).;true;true
1;false,true;publication(F, B).;true;true
1;fa

Results
   AUC ROC   = 0.864498
   AUC PR    = 0.068086
   CLL	      = -0.602732
   Precision = 0.065606 at threshold = 0.5
   Recall    = 0.942857
   F1        = 0.122677


Total learning time: 2.231 seconds
Total inference time: 1.064 seconds
AUC ROC: 0.864498


Results scoring model
   AUC ROC   = 0.990358
   AUC PR    = 0.991063
   CLL	      = -0.182718
   Precision = 1.0 at threshold = 0.5
   Recall    = 0.878788
   F1        = 0.935484


Total scoring time: 0.514 seconds
Refined model CLL: -0.182718


******************************************
Best model found
******************************************
Results
   AUC ROC   = 0.864498
   AUC PR    = 0.068086
   CLL	      = -0.602732
   Precision = 0.065606 at threshold = 0.5
   Recall    = 0.942857
   F1        = 0.122677


Total learning time: 4.387 seconds
Total inference time: 1.337 seconds
AUC ROC: 0.864498
Total revision time: 8.86
Best scored revision CLL: -0.182718


[<boostsrl.boostsrl.train object at 0x7fabad733160>, {'AU

WILL Produced-Tree #1
% FOR advisedby(A, B):
%   if ( student(B), student(A) )
%   then if ( publication(C, A), publication(C, B) )
%   | then return 0;  // std dev = 0,000, 0,000 (wgt'ed) examples reached here.
%   | else return -0.1418510649004878;  // std dev = 0,000, 12,000 (wgt'ed) examples reached here.  /* #neg=12 */
%   else return 0.6581489350995123;  // std dev = 1,789, 20,000 (wgt'ed) examples reached here.  /* #neg=4 #pos=16 */
WILL Produced-Tree #2
% FOR advisedby(A, B):
%   if ( student(B), student(A) )
%   then if ( publication(C, A), publication(C, B) )
%   | then return 0;  // std dev = 0,000, 0,000 (wgt'ed) examples reached here.
%   | else return -0.12544463852839138;  // std dev = 5,27e-09, 12,000 (wgt'ed) examples reached here.  /* #neg=12 */
%   else return 0.5580193346218546;  // std dev = 1,789, 20,000 (wgt'ed) examples reached here.  /* #neg=4 #pos=16 */
WILL Produced-Tree #3
% FOR advisedby(A, B):
%   if ( professor(B), student(A) )
%   then if ( publication(C

Results
   AUC ROC   = 0.906615
   AUC PR    = 0.105009
   CLL	      = -0.63512
   Precision = 0.061706 at threshold = 0.5
   Recall    = 0.971429
   F1        = 0.116041


Total learning time: 4.049 seconds
Total inference time: 1.019 seconds
AUC ROC: 0.906615


Results scoring model
   AUC ROC   = 0.988281
   AUC PR    = 0.989436
   CLL	      = -0.217022
   Precision = 0.9375 at threshold = 0.5
   Recall    = 0.9375
   F1        = 0.9375


Total scoring time: 0.483 seconds
Refined model CLL: -0.217022


Refining iteration 2
********************************
Candidate for revision
0;;advisedby(A, B) :- student(B), student(A).;true;true
0;false;projectmember(C, B), ta(D, A, E).;true;true
0;false,false;ta(F, A, G).;true;true
0;false,false,true;ta(H, I, G), publication(J, I).;true;true
1;;advisedby(A, B) :- student(B), student(A).;true;true
1;false;projectmember(C, B), ta(D, A, E).;true;true
1;false,false;ta(F, A, G).;true;true
1;false,false,true;ta(H, I, G), publication(J, I).;true;true


Results
   AUC ROC   = 0.885116
   AUC PR    = 0.076904
   CLL	      = -0.624854
   Precision = 0.062157 at threshold = 0.5
   Recall    = 0.971429
   F1        = 0.116838


Total learning time: 2.513 seconds
Total inference time: 1.154 seconds
AUC ROC: 0.885116


Results scoring model
   AUC ROC   = 0.984375
   AUC PR    = 0.98527
   CLL	      = -0.228788
   Precision = 0.9375 at threshold = 0.5
   Recall    = 0.9375
   F1        = 0.9375


Total scoring time: 0.483 seconds
Refined model CLL: -0.228788


******************************************
Best model found
******************************************
Results
   AUC ROC   = 0.906615
   AUC PR    = 0.105009
   CLL	      = -0.63512
   Precision = 0.061706 at threshold = 0.5
   Recall    = 0.971429
   F1        = 0.116041


Total learning time: 4.049 seconds
Total inference time: 1.019 seconds
AUC ROC: 0.906615
Total revision time: 8.753
Best scored revision CLL: -0.217022


[<boostsrl.boostsrl.train object at 0x7fabad755f28>, {'AUC 

In [14]:
refine

['0;;workedunder00(A,B):- director01(B), actor01(A);true;false',
 '0;true;movie02(C, A), movie03(C, B).;false;false',
 '1;;workedunder10(A,B):- director11(B), actor11(A);true;false',
 '1;true;movie12(C, A), movie13(C, B).;false;false',
 '2;;workedunder20(A,B):- director21(B), actor21(A);true;false',
 '2;true;movie22(C, A), movie23(C, B).;false;false',
 '3;;workedunder30(A,B):- actor31(A), director31(B);true;false',
 '3;true;movie32(C, A), movie33(C, B).;false;false',
 '4;;workedunder40(A,B):- director41(B), actor41(A);true;false',
 '4;true;movie42(C, A), movie43(C, B).;false;false',
 '5;;workedunder50(A,B):- actor51(A), director51(B);true;false',
 '5;true;movie52(C, A), movie53(C, B).;false;false',
 '6;;workedunder60(A,B):- director61(B), actor61(A);true;false',
 '6;true;movie62(C, A), movie63(C, B).;false;false',
 '7;;workedunder70(A,B):- actor71(A), director71(B);true;false',
 '7;true;movie72(C, A), movie73(C, B).;false;false',
 '8;;workedunder80(A,B):- director81(B), actor81(A);true

## Aleatório

In [9]:
# train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset, 0)
# uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773) #facts, pos, neg


start = time.time()
res = genetic(new_src_struct, target, source, train_pos, 
                                         train_neg, train_facts, kb_source, kb_target, pred_target,
                                          NUM_GEN=14, pop_size=30, revision='random')
print("FIM DO GROOT GENETIC: ", time.time()-start)

GENERATION:  0
MELHOR RESULTADO:  -0.38296325
BEST: {'m_auc_pr': 0.38296325, 'm_auc_roc': 0.93727275, 'm_cll': -0.27942524999999996, 'm_rec': 0.67051975, 'm_pred': 0.46463987500000004, 'm_f1': 0.34964175, 's_auc_pr': 0.08505931323310514, 's_auc_roc': 0.00465756649845174, 's_cll': 0.026465637781612227, 's_rec': 0.3301591929602862, 's_prec': 0.22157014338603784, 's_f1': 0.103342275241972}
GENERATION:  1
MELHOR RESULTADO:  -0.38296325
BEST: {'m_auc_pr': 0.38296325, 'm_auc_roc': 0.93727275, 'm_cll': -0.28039349999999996, 'm_rec': 1.0, 'm_pred': 0.337133625, 'm_f1': 0.2935665, 's_auc_pr': 0.08505931323310514, 's_auc_roc': 0.00465756649845174, 's_cll': 0.03815803569695378, 's_rec': 0.0, 's_prec': 0.16703793298824185, 's_f1': 0.07261105873246858}
GENERATION:  2
MELHOR RESULTADO:  -0.38296325
BEST: {'m_auc_pr': 0.38296325, 'm_auc_roc': 0.93727275, 'm_cll': -0.2860815, 'm_rec': 1.0, 'm_pred': 0.337133625, 'm_f1': 0.2935665, 's_auc_pr': 0.08505931323310514, 's_auc_roc': 0.00465756649845174, 's_c

In [ ]:
res

In [ ]:
def get_k_best_individuals(population, k):
    best_ind = []
    all_fitness = []
    for i in range(0, len(population)):
        all_fitness.append((population[i].fitness.values[0], i))
    
    sorted_ind = sorted(all_fitness, reverse=True, key=lambda tup: tup[0])
    for i in range(0, k):
        best_ind.append(population[sorted_ind[i][1]])
    return best_ind

In [ ]:
final_results = {}
final_results[f'{source}->{target}'] = []
final_results[f'{source}->{target}'].append(res)
best = final_results[f'{source}->{target}'][0]
for i in final_results[f'{source}->{target}']:
    if i[1][-1] > best[1][-1]:
        best = i

In [ ]:
# final_results[f'groot_test:{source}->{target}'] = []
# final_results[f'groot_test_rev:{source}->{target}'] = []
# final_results[f'inf_res:{source}->{target}'] = []
# final_results[f'inf_res_rev:{source}->{target}'] = []

# uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773)
# train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset, 1)
# tr_train_pos = []
# tr_train_neg = []
# tr_train_facts = []

# for i in range(0, len(train_pos)):
#     tr_train_pos.extend(train_pos[i])
#     tr_train_neg.extend(train_neg[i])
#     tr_train_facts.extend(train_facts[i])

# train = [tr_train_pos, tr_train_neg, tr_train_facts]
test = [test_pos, test_neg, test_facts]

# individuals = get_k_best_individuals(res[0].population, 3)

n_ind = 1
for individual in individuals:
    print("INDIVIDUO ", n_ind)
    refine, transfer = get_refine_transfer(individual, source, target, 'imdb', 'uwcse')
    for i in range(len(train_pos)):
        ttrain = [train_pos[i], train_neg[i][:len(train_pos[i])], train_facts[i]]
        res_ = test_refine_transfer(kb_target, target, refine, transfer, ttrain, test)
        print(res_)
#         final_results[f'groot_test:{source}->{target}'].append(res)
#         import os
    #     thisFile = f'boostsrl/best/test/results_{target.replace('recursion_', '')}.db'
    #     thisFile = f'boostsrl/best/test/results_{target}.db'
    #     base = os.path.splitext(thisFile)[0]
    #     os.rename(thisFile, base + ".txt")
    #     tt = open('boostsrl/best/test/results_advisedby.txt', 'r').readlines()
    #     final = []
    #     for i in tt:
    #         final.append(i.replace('\n', ''))
    #     final_results[f'inf_res:{source}->{target}'].append(final)
        print("===========================================================================")
#         break

    n_ind += 1

In [66]:
cll = [-0.371804,-0.371804,-0.371804]
pr = [0.942857,0.942857,0.942857]
roc = [0.942857,0.942857,0.942857]


import numpy as np
print("CLL: ", np.mean(cll), np.std(cll))
print("ROC: ", np.mean(roc), np.std(roc))
print("PR: ", np.mean(pr), np.std(pr))

CLL:  -0.371804 0.0
ROC:  0.942857 0.0
PR:  0.942857 0.0


In [24]:
final_results[f'brkga_{source}->{target}'] = []
final_results[f'inf_res_brkga:{source}->{target}'] = []
final_results[f'brkga_var_{source}->{target}'] = []
final_results[f'inf_res_brkga_var:{source}->{target}'] = []
save_results(final_results, source, target, 'imdb', 'uwcse')

In [25]:
final_results[f'groot_test:{source}->{target}']

[({'AUC ROC': 0.942857,
   'AUC PR': 0.897436,
   'CLL': -0.371804,
   'Precision': [0.897436, 0.5],
   'Recall': 1.0,
   'F1': 0.945946},
  <boostsrl.boostsrl.test at 0x13b4a68e0>),
 ({'AUC ROC': 0.942857,
   'AUC PR': 0.897436,
   'CLL': -0.371804,
   'Precision': [0.897436, 0.5],
   'Recall': 1.0,
   'F1': 0.945946},
  <boostsrl.boostsrl.test at 0x1077c8c40>),
 ({'AUC ROC': 0.942857,
   'AUC PR': 0.897436,
   'CLL': -0.371804,
   'Precision': [0.897436, 0.5],
   'Recall': 1.0,
   'F1': 0.945946},
  <boostsrl.boostsrl.test at 0x13b4a6d00>)]

## Apenas guiado

In [127]:
start = time.time()
res_g = genetic(new_src_struct, target, source, train_pos, 
                                         train_neg, train_facts, kb_source, kb_target, pred_target,
                                          NUM_GEN=14, pop_size=30)
print("FIM DO GROOT GENETIC: ", time.time()-start)

GENERATION:  0
MELHOR RESULTADO:  -1.092970248936728
BEST: {'m_auc_pr': 0.17426725, 'm_auc_roc': 0.9099392500000001, 'm_cll': -0.6054705, 'm_rec': 1.0, 'm_pred': 0.285546, 'm_f1': 0.1320275, 's_auc_pr': 0.052463605505983855, 's_auc_roc': 0.0057985571211724716, 's_cll': 0.0503632170959124, 's_rec': 0.0, 's_prec': 0.21497987868344795, 's_f1': 0.03626637711089984}
GENERATION:  1
MELHOR RESULTADO:  -1.092970248936728
BEST: {'m_auc_pr': 0.17426725, 'm_auc_roc': 0.9099392500000001, 'm_cll': -0.6054705, 'm_rec': 1.0, 'm_pred': 0.285546, 'm_f1': 0.1320275, 's_auc_pr': 0.052463605505983855, 's_auc_roc': 0.0057985571211724716, 's_cll': 0.0503632170959124, 's_rec': 0.0, 's_prec': 0.21497987868344795, 's_f1': 0.03626637711089984}
GENERATION:  2
MELHOR RESULTADO:  -1.1247142571042767
BEST: {'m_auc_pr': 0.17426725, 'm_auc_roc': 0.9099392500000001, 'm_cll': -0.66105425, 'm_rec': 1.0, 'm_pred': 0.285546, 'm_f1': 0.1320275, 's_auc_pr': 0.052463605505983855, 's_auc_roc': 0.0057985571211724716, 's_cll': 

In [142]:
res_g

(<src.s_genetic.population.Population at 0x125537580>,
 [-1.092970248936728,
  -1.092970248936728,
  -1.1247142571042767,
  -1.1278426691961794,
  -1.1588017600153024,
  -1.1588017600153024,
  -1.1588017600153024,
  -1.1588017600153024,
  -1.1588017600153024,
  -1.1588017600153024,
  -1.1588017600153024,
  -1.1588017600153024,
  -1.1588017600153024],
 [{'m_auc_pr': 0.17426725,
   'm_auc_roc': 0.9099392500000001,
   'm_cll': -0.6054705,
   'm_rec': 1.0,
   'm_pred': 0.285546,
   'm_f1': 0.1320275,
   's_auc_pr': 0.052463605505983855,
   's_auc_roc': 0.0057985571211724716,
   's_cll': 0.0503632170959124,
   's_rec': 0.0,
   's_prec': 0.21497987868344795,
   's_f1': 0.03626637711089984},
  {'m_auc_pr': 0.17426725,
   'm_auc_roc': 0.9099392500000001,
   'm_cll': -0.6054705,
   'm_rec': 1.0,
   'm_pred': 0.285546,
   'm_f1': 0.1320275,
   's_auc_pr': 0.052463605505983855,
   's_auc_roc': 0.0057985571211724716,
   's_cll': 0.0503632170959124,
   's_rec': 0.0,
   's_prec': 0.21497987868344795

In [134]:
# final_results[f'groot_test:{source}->{target}'] = []
# final_results[f'groot_test_rev:{source}->{target}'] = []
# final_results[f'inf_res:{source}->{target}'] = []
# final_results[f'inf_res_rev:{source}->{target}'] = []

# uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773)
# train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset, 1)
# tr_train_pos = []
# tr_train_neg = []
# tr_train_facts = []

# for i in range(0, len(train_pos)):
#     tr_train_pos.extend(train_pos[i])
#     tr_train_neg.extend(train_neg[i])
#     tr_train_facts.extend(train_facts[i])

# train = [tr_train_pos, tr_train_neg, tr_train_facts]
test = [test_pos, test_neg, test_facts]

individuals = get_k_best_individuals(res_g[0].population, 3)

n_ind = 1
for individual in individuals:
    print("INDIVIDUO ", n_ind)
    refine, transfer = get_refine_transfer(individual, source, target, 'imdb', 'uwcse')
    for i in range(len(train_pos)):
        ttrain = [train_pos[i], train_neg[i][:len(train_pos[i])], train_facts[i]]
        res_ = test_refine_transfer(kb_target, target, refine, transfer, ttrain, test)
        print(res_)
#         final_results[f'groot_test:{source}->{target}'].append(res)
#         import os
    #     thisFile = f'boostsrl/best/test/results_{target.replace('recursion_', '')}.db'
    #     thisFile = f'boostsrl/best/test/results_{target}.db'
    #     base = os.path.splitext(thisFile)[0]
    #     os.rename(thisFile, base + ".txt")
    #     tt = open('boostsrl/best/test/results_advisedby.txt', 'r').readlines()
    #     final = []
    #     for i in tt:
    #         final.append(i.replace('\n', ''))
    #     final_results[f'inf_res:{source}->{target}'].append(final)
        print("===========================================================================")
#         break

    n_ind += 1

INDIVIDUO  1
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.7572, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x127433970>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.804689, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x127433a90>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.722383, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x12843c6d0>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.657902, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x127433a90>)
INDIVIDUO  2
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.7572, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x12843c910>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.804689, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boost

In [67]:
cll = [-0.427185,-0.427185,-0.427185]
pr = [0.897436,0.897436,0.897436]
roc = [0.942857,0.942857,0.942857]


import numpy as np
print("CLL: ", np.mean(cll), np.std(cll))
print("ROC: ", np.mean(roc), np.std(roc))
print("PR: ", np.mean(pr), np.std(pr))

CLL:  -0.427185 0.0
ROC:  0.942857 0.0
PR:  0.897436 0.0


In [33]:
final_results[f'brkga_{source}->{target}'] = []
final_results[f'inf_res_brkga:{source}->{target}'] = []
final_results[f'brkga_var_{source}->{target}'] = []
final_results[f'inf_res_brkga_var:{source}->{target}'] = []
save_results(final_results, source, target, 'imdb', 'uwcse')

In [34]:
final_results[f'groot_test:{source}->{target}']

[({'AUC ROC': 0.942857,
   'AUC PR': 0.897436,
   'CLL': -0.427185,
   'Precision': [0.897436, 0.5],
   'Recall': 1.0,
   'F1': 0.945946},
  <boostsrl.boostsrl.test at 0x13b3b9a60>),
 ({'AUC ROC': 0.942857,
   'AUC PR': 0.897436,
   'CLL': -0.427185,
   'Precision': [0.897436, 0.5],
   'Recall': 1.0,
   'F1': 0.945946},
  <boostsrl.boostsrl.test at 0x13b3b9e20>),
 ({'AUC ROC': 0.942857,
   'AUC PR': 0.897436,
   'CLL': -0.427185,
   'Precision': [0.897436, 0.5],
   'Recall': 1.0,
   'F1': 0.945946},
  <boostsrl.boostsrl.test at 0x13b3b98e0>)]

## Mixed

In [135]:
start = time.time()
res = genetic(new_src_struct, target, source, train_pos, 
                                         train_neg, train_facts, kb_source, kb_target, pred_target,
                                          NUM_GEN=14, pop_size=30)
print("FIM DO GROOT GENETIC: ", time.time()-start)

GENERATION:  0
MELHOR RESULTADO:  -1.0718735101174777
BEST: {'m_auc_pr': 0.17426725, 'm_auc_roc': 0.9099392500000001, 'm_cll': -0.566501, 'm_rec': 1.0, 'm_pred': 0.285546, 'm_f1': 0.1320275, 's_auc_pr': 0.052463605505983855, 's_auc_roc': 0.0057985571211724716, 's_cll': 0.04964464285801641, 's_rec': 0.0, 's_prec': 0.21497987868344795, 's_f1': 0.03626637711089984}
GENERATION:  1
MELHOR RESULTADO:  -1.0718735101174777
BEST: {'m_auc_pr': 0.17426725, 'm_auc_roc': 0.9099392500000001, 'm_cll': -0.566501, 'm_rec': 1.0, 'm_pred': 0.285546, 'm_f1': 0.1320275, 's_auc_pr': 0.052463605505983855, 's_auc_roc': 0.0057985571211724716, 's_cll': 0.04964464285801641, 's_rec': 0.0, 's_prec': 0.21497987868344795, 's_f1': 0.03626637711089984}
GENERATION:  2
MELHOR RESULTADO:  -1.0718735101174777
BEST: {'m_auc_pr': 0.17426725, 'm_auc_roc': 0.9099392500000001, 'm_cll': -0.566501, 'm_rec': 1.0, 'm_pred': 0.285546, 'm_f1': 0.1320275, 's_auc_pr': 0.052463605505983855, 's_auc_roc': 0.0057985571211724716, 's_cll': 

In [141]:
res

(<src.s_genetic.population.Population at 0x12781b4c0>,
 [-1.0718735101174777,
  -1.0718735101174777,
  -1.0718735101174777,
  -1.0889420067196531,
  -1.0889420067196531,
  -1.1057602455554336,
  -1.1057602455554336,
  -1.1057602455554336,
  -1.1057602455554336,
  -1.1057602455554336,
  -1.1057602455554336,
  -1.1057602455554336,
  -1.1057602455554336,
  -1.1057602455554336],
 [{'m_auc_pr': 0.17426725,
   'm_auc_roc': 0.9099392500000001,
   'm_cll': -0.566501,
   'm_rec': 1.0,
   'm_pred': 0.285546,
   'm_f1': 0.1320275,
   's_auc_pr': 0.052463605505983855,
   's_auc_roc': 0.0057985571211724716,
   's_cll': 0.04964464285801641,
   's_rec': 0.0,
   's_prec': 0.21497987868344795,
   's_f1': 0.03626637711089984},
  {'m_auc_pr': 0.17426725,
   'm_auc_roc': 0.9099392500000001,
   'm_cll': -0.566501,
   'm_rec': 1.0,
   'm_pred': 0.285546,
   'm_f1': 0.1320275,
   's_auc_pr': 0.052463605505983855,
   's_auc_roc': 0.0057985571211724716,
   's_cll': 0.04964464285801641,
   's_rec': 0.0,
   's_p

In [140]:
final_results = {}
final_results[f'{source}->{target}'] = []
final_results[f'{source}->{target}'].append(res)
best = final_results[f'{source}->{target}'][0]
for i in final_results[f'{source}->{target}']:
    if i[1][-1] > best[1][-1]:
        best = i

In [143]:
# final_results[f'groot_test:{source}->{target}'] = []
# final_results[f'groot_test_rev:{source}->{target}'] = []
# final_results[f'inf_res:{source}->{target}'] = []
# final_results[f'inf_res_rev:{source}->{target}'] = []

# uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773)
# train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset, 1)
# tr_train_pos = []
# tr_train_neg = []
# tr_train_facts = []

# for i in range(0, len(train_pos)):
#     tr_train_pos.extend(train_pos[i])
#     tr_train_neg.extend(train_neg[i])
#     tr_train_facts.extend(train_facts[i])

# train = [tr_train_pos, tr_train_neg, tr_train_facts]
test = [test_pos, test_neg, test_facts]

individuals = get_k_best_individuals(res[0].population, 3)

n_ind = 1
for individual in individuals:
    print("INDIVIDUO ", n_ind)
    refine, transfer = get_refine_transfer(individual, source, target, 'imdb', 'uwcse')
    for i in range(len(train_pos)):
        ttrain = [train_pos[i], train_neg[i][:len(train_pos[i])], train_facts[i]]
        res_ = test_refine_transfer(kb_target, target, refine, transfer, ttrain, test)
        print(res_)
#         final_results[f'groot_test:{source}->{target}'].append(res)
#         import os
    #     thisFile = f'boostsrl/best/test/results_{target.replace('recursion_', '')}.db'
    #     thisFile = f'boostsrl/best/test/results_{target}.db'
    #     base = os.path.splitext(thisFile)[0]
    #     os.rename(thisFile, base + ".txt")
    #     tt = open('boostsrl/best/test/results_advisedby.txt', 'r').readlines()
    #     final = []
    #     for i in tt:
    #         final.append(i.replace('\n', ''))
    #     final_results[f'inf_res:{source}->{target}'].append(final)
        print("===========================================================================")
#         break

    n_ind += 1

INDIVIDUO  1
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.657272, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x1269481c0>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.690449, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x126948e50>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.632716, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x1269485b0>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.577927, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x1269481c0>)
INDIVIDUO  2
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.657272, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x1269486a0>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.690449, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <b

# Crossover modificado

## sem revisão

In [17]:
start = time.time()
res_cm_r = genetic(new_src_struct, target, source, train_pos, 
                                         train_neg, train_facts, kb_source, kb_target, pred_target,
                                          NUM_GEN=14, pop_size=30)
print("FIM DO GROOT GENETIC: ", time.time()-start)

GENERATION:  0
MELHOR RESULTADO:  -1.1094503367727304
BEST: {'m_auc_pr': 0.38296325, 'm_auc_roc': 0.93727275, 'm_cll': -0.5936327499999999, 'm_rec': 1.0, 'm_pred': 0.285546, 'm_f1': 0.1320275, 's_auc_pr': 0.08505931323310514, 's_auc_roc': 0.00465756649845174, 's_cll': 0.05434531231566802, 's_rec': 0.0, 's_prec': 0.21497987868344795, 's_f1': 0.03626637711089984}
GENERATION:  1
MELHOR RESULTADO:  -1.1094503367727304
BEST: {'m_auc_pr': 0.38296325, 'm_auc_roc': 0.93727275, 'm_cll': -0.5936327499999999, 'm_rec': 1.0, 'm_pred': 0.285546, 'm_f1': 0.1320275, 's_auc_pr': 0.08505931323310514, 's_auc_roc': 0.00465756649845174, 's_cll': 0.05434531231566802, 's_rec': 0.0, 's_prec': 0.21497987868344795, 's_f1': 0.03626637711089984}
GENERATION:  2
MELHOR RESULTADO:  -1.1094503367727304
BEST: {'m_auc_pr': 0.38296325, 'm_auc_roc': 0.93727275, 'm_cll': -0.5936327499999999, 'm_rec': 1.0, 'm_pred': 0.285546, 'm_f1': 0.1320275, 's_auc_pr': 0.08505931323310514, 's_auc_roc': 0.00465756649845174, 's_cll': 0.0

In [18]:
res_cm_r

(<src.s_genetic.population.Population at 0x1254cf340>,
 [-1.1094503367727304,
  -1.1094503367727304,
  -1.1094503367727304,
  -1.1094503367727304,
  -1.1228929443008469,
  -1.1269619649271942,
  -1.1269619649271942,
  -1.1269619649271942,
  -1.1269619649271942,
  -1.1407769139821171,
  -1.1622578391295646,
  -1.1622578391295646,
  -1.1622578391295646,
  -1.1622578391295646,
  -1.1627357455301806],
 [{'m_auc_pr': 0.38296325,
   'm_auc_roc': 0.93727275,
   'm_cll': -0.5936327499999999,
   'm_rec': 1.0,
   'm_pred': 0.285546,
   'm_f1': 0.1320275,
   's_auc_pr': 0.08505931323310514,
   's_auc_roc': 0.00465756649845174,
   's_cll': 0.05434531231566802,
   's_rec': 0.0,
   's_prec': 0.21497987868344795,
   's_f1': 0.03626637711089984},
  {'m_auc_pr': 0.38296325,
   'm_auc_roc': 0.93727275,
   'm_cll': -0.5936327499999999,
   'm_rec': 1.0,
   'm_pred': 0.285546,
   'm_f1': 0.1320275,
   's_auc_pr': 0.08505931323310514,
   's_auc_roc': 0.00465756649845174,
   's_cll': 0.05434531231566802,
   

In [19]:
# final_results[f'groot_test:{source}->{target}'] = []
# final_results[f'groot_test_rev:{source}->{target}'] = []
# final_results[f'inf_res:{source}->{target}'] = []
# final_results[f'inf_res_rev:{source}->{target}'] = []

# uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773)
# train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset, 1)
# tr_train_pos = []
# tr_train_neg = []
# tr_train_facts = []

# for i in range(0, len(train_pos)):
#     tr_train_pos.extend(train_pos[i])
#     tr_train_neg.extend(train_neg[i])
#     tr_train_facts.extend(train_facts[i])

# train = [tr_train_pos, tr_train_neg, tr_train_facts]
test = [test_pos, test_neg, test_facts]

individuals = get_k_best_individuals(res_cm_r[0].population, 3)

n_ind = 1
for individual in individuals:
    print("INDIVIDUO ", n_ind)
    refine, transfer = get_refine_transfer(individual, source, target, 'imdb', 'uwcse')
    for i in range(len(train_pos)):
        ttrain = [train_pos[i], train_neg[i][:len(train_pos[i])], train_facts[i]]
#         res_ = test_refine_transfer(kb_target, target, refine, transfer, ttrain, test)
        res_ = test_tree_b(source, target, kb_source, kb_target, refine, ttrain, test, transfer)
        print(res_)
#         final_results[f'groot_test:{source}->{target}'].append(res)
#         import os
    #     thisFile = f'boostsrl/best/test/results_{target.replace('recursion_', '')}.db'
    #     thisFile = f'boostsrl/best/test/results_{target}.db'
    #     base = os.path.splitext(thisFile)[0]
    #     os.rename(thisFile, base + ".txt")
    #     tt = open('boostsrl/best/test/results_advisedby.txt', 'r').readlines()
    #     final = []
    #     for i in tt:
    #         final.append(i.replace('\n', ''))
    #     final_results[f'inf_res:{source}->{target}'].append(final)
        print("===========================================================================")
#         break

    n_ind += 1

INDIVIDUO  1
******************************************
Performing Parameter Learning
******************************************
Refine
0;;workedunder00(A,B):- director01(B), actor01(A);true;false
0;true;movie02(C, A), movie03(C, B).;false;false
1;;workedunder10(A,B):- director11(B), actor11(A);true;false
1;true;movie12(C, A), movie13(C, B).;false;false
2;;workedunder20(A,B):- director21(B), actor21(A);true;false
2;true;movie22(C, A), movie23(C, B).;false;false
3;;workedunder30(A,B):- actor31(A), director31(B);true;false
3;true;movie32(C, A), movie33(C, B).;false;false
4;;workedunder40(A,B):- director41(B), actor41(A);true;false
4;true;movie42(C, A), movie43(C, B).;false;false
5;;workedunder50(A,B):- actor51(A), director51(B);true;false
5;true;movie52(C, A), movie53(C, B).;false;false
6;;workedunder60(A,B):- director61(B), actor61(A);true;false
6;true;movie62(C, A), movie63(C, B).;false;false
7;;workedunder70(A,B):- actor71(A), director71(B);true;false
7;true;movie72(C, A), movie73(C, 

Results
   AUC ROC   = 0.90215
   AUC PR    = 0.089744
   CLL	      = -0.761715
   Precision = 0.036881 at threshold = 0.5
   Recall    = 1.0
   F1        = 0.071138


Total learning time: 0.81 seconds
Total inference time: 1.033 seconds
AUC ROC: 0.90215


Results scoring model
   AUC ROC   = 0.925
   AUC PR    = 0.869565
   CLL	      = -0.272928
   Precision = 0.869565 at threshold = 0.5
   Recall    = 1.0
   F1        = 0.930233


Total scoring time: 0.441 seconds
Parameter learned model CLL: -0.272928


Structure after Parameter Learning
[['advisedby(A, B)', {'': 'student(B), student(A)', 'true': 'publication(C, A), publication(C, B)'}, {'true,true': [0.0, 3, 0], 'true,false': [0.0, 14, 0], 'false': [1.615, 3, 20]}], ['advisedby(A, B)', {'': 'student(B), student(A)', 'true': 'publication(C, A), publication(C, B)'}, {'true,true': [0.0, 3, 0], 'true,false': [5.27e-09, 14, 0], 'false': [1.615, 3, 20]}], ['advisedby(A, B)', {'': 'student(B), student(A)', 'true': 'publication(C, A), publ

Results
   AUC ROC   = 0.827256
   AUC PR    = 0.099552
   CLL	      = -0.946288
   Precision = 0.037158 at threshold = 0.5
   Recall    = 0.971429
   F1        = 0.071579


Total learning time: 4.603 seconds
Total inference time: 1.269 seconds
AUC ROC: 0.827256


Results scoring model
   AUC ROC   = 1.0
   AUC PR    = 1.0
   CLL	      = -0.145401
   Precision = 1.0 at threshold = 0.5
   Recall    = 1.0
   F1        = 1.0


Total scoring time: 0.479 seconds
Refined model CLL: -0.145401


Refining iteration 2
********************************
Candidate for revision
0;;advisedby(A, B) :- student(B), student(A).;true;true
0;true;publication(C, A), publication(C, B).;true;true
0;false;ta(D, A, E), publication(F, A).;true;true
1;;advisedby(A, B) :- student(B), student(A).;true;true
1;true;publication(C, A), publication(C, B).;true;true
1;true,false;publication(D, B).;true;true
1;true,false,true;inphase(A, E), inphase(B, E).;true;true
1;false;ta(F, A, G), publication(H, A).;true;true
1;false,

Results
   AUC ROC   = 0.815176
   AUC PR    = 0.096136
   CLL	      = -0.948694
   Precision = 0.037158 at threshold = 0.5
   Recall    = 0.971429
   F1        = 0.071579


Total learning time: 2.394 seconds
Total inference time: 1.264 seconds
AUC ROC: 0.815176


Results scoring model
   AUC ROC   = 1.0
   AUC PR    = 1.0
   CLL	      = -0.141984
   Precision = 1.0 at threshold = 0.5
   Recall    = 1.0
   F1        = 1.0


Total scoring time: 0.457 seconds
Refined model CLL: -0.141984


Refining iteration 3
********************************
Candidate for revision
0;;advisedby(A, B) :- student(B), student(A).;true;true
0;true;publication(C, A), publication(C, B).;true;true
0;false;ta(D, A, E), publication(F, A).;true;true
1;;advisedby(A, B) :- student(B), student(A).;true;true
1;true;publication(C, A), publication(C, B).;true;true
1;true,false;publication(D, B).;true;true
1;true,false,true;inphase(A, E), inphase(B, E).;true;true
1;false;ta(F, A, G), publication(H, A).;true;true
1;false,

Results
   AUC ROC   = 0.815176
   AUC PR    = 0.096136
   CLL	      = -0.948694
   Precision = 0.037158 at threshold = 0.5
   Recall    = 0.971429
   F1        = 0.071579


Total learning time: 1.548 seconds
Total inference time: 1.282 seconds
AUC ROC: 0.815176


Results scoring model
   AUC ROC   = 1.0
   AUC PR    = 1.0
   CLL	      = -0.141984
   Precision = 1.0 at threshold = 0.5
   Recall    = 1.0
   F1        = 1.0


Total scoring time: 0.501 seconds
Refined model CLL: -0.141984


******************************************
Best model found
******************************************
Results
   AUC ROC   = 0.815176
   AUC PR    = 0.096136
   CLL	      = -0.948694
   Precision = 0.037158 at threshold = 0.5
   Recall    = 0.971429
   F1        = 0.071579


Total learning time: 2.394 seconds
Total inference time: 1.264 seconds
AUC ROC: 0.815176
Total revision time: 11.233
Best scored revision CLL: -0.141984


[<boostsrl.boostsrl.train object at 0x12587f0a0>, {'AUC ROC': 0.815176, 'AU

...

Results
   AUC ROC   = 0.864498
   AUC PR    = 0.068086
   CLL	      = -0.898645
   Precision = 0.036954 at threshold = 0.5
   Recall    = 0.942857
   F1        = 0.071121


Total learning time: 1.816 seconds
Total inference time: 1.106 seconds
AUC ROC: 0.864498


Results scoring model
   AUC ROC   = 0.990358
   AUC PR    = 0.991063
   CLL	      = -0.182718
   Precision = 1.0 at threshold = 0.5
   Recall    = 0.878788
   F1        = 0.935484


Total scoring time: 0.469 seconds
Refined model CLL: -0.182718


******************************************
Best model found
******************************************
Results
   AUC ROC   = 0.864498
   AUC PR    = 0.068086
   CLL	      = -0.898645
   Precision = 0.036954 at threshold = 0.5
   Recall    = 0.942857
   F1        = 0.071121


Total learning time: 4.215 seconds
Total inference time: 1.051 seconds
AUC ROC: 0.864498
Total revision time: 8.203
Best scored revision CLL: -0.182718


[<boostsrl.boostsrl.train object at 0x12587f0a0>, {'AUC 

WILL Produced-Tree #1
% FOR advisedby(A, B):
%   if ( student(B), student(A) )
%   then if ( publication(C, A), publication(C, B) )
%   | then return 0;  // std dev = 0,000, 0,000 (wgt'ed) examples reached here.
%   | else return -0.1418510649004878;  // std dev = 0,000, 12,000 (wgt'ed) examples reached here.  /* #neg=12 */
%   else return 0.6581489350995123;  // std dev = 1,789, 20,000 (wgt'ed) examples reached here.  /* #neg=4 #pos=16 */
WILL Produced-Tree #2
% FOR advisedby(A, B):
%   if ( student(B), student(A) )
%   then if ( publication(C, A), publication(C, B) )
%   | then return 0;  // std dev = 0,000, 0,000 (wgt'ed) examples reached here.
%   | else return -0.12544463852839138;  // std dev = 5,27e-09, 12,000 (wgt'ed) examples reached here.  /* #neg=12 */
%   else return 0.5580193346218546;  // std dev = 1,789, 20,000 (wgt'ed) examples reached here.  /* #neg=4 #pos=16 */
WILL Produced-Tree #3
% FOR advisedby(A, B):
%   if ( student(B), student(A) )
%   then if ( publication(C, 

WILL Produced-Tree #1
% FOR advisedby(A, B):
%   if ( student(B), student(A) )
%   then return -0.1418510649004878;  // std dev = 0,000, 12,000 (wgt'ed) examples reached here.  /* #neg=12 */
%   else if ( ta(C, A, D) )
%   | then if ( ta(E, F, D), publication(G, F) )
%   | | then return 0.8581489350995123;  // std dev = 0,000, 6,000 (wgt'ed) examples reached here.  /* #pos=6 */
%   | | else if ( projectmember(H, B) )
%   | | | then return 0.19148226843284552;  // std dev = 0,816, 3,000 (wgt'ed) examples reached here.  /* #neg=2 #pos=1 */
%   | | | else return 0.4581489350995122;  // std dev = 1,095, 5,000 (wgt'ed) examples reached here.  /* #neg=2 #pos=3 */
%   | else return 0.8581489350995123;  // std dev = 0,000, 6,000 (wgt'ed) examples reached here.  /* #pos=6 */
WILL Produced-Tree #2
% FOR advisedby(A, B):
%   if ( student(B), student(A) )
%   then return -0.12544463852839138;  // std dev = 5,27e-09, 12,000 (wgt'ed) examples reached here.  /* #neg=12 */
%   else if ( projectmember(

Results scoring model
   AUC ROC   = 0.990234
   AUC PR    = 0.991389
   CLL	      = -0.213029
   Precision = 0.9375 at threshold = 0.5
   Recall    = 0.9375
   F1        = 0.9375


Total scoring time: 0.525 seconds
Refined model CLL: -0.213029


Refining iteration 2
********************************
Candidate for revision
0;;advisedby(A, B) :- student(B), student(A).;true;true
0;false;ta(C, A, D).;true;true
0;false,true;ta(E, F, D), publication(G, F).;true;true
1;;advisedby(A, B) :- student(B), student(A).;true;true
1;false;projectmember(C, B), ta(D, A, E).;true;true
1;false,false;ta(F, A, G).;true;true
1;false,false,true;ta(H, I, G), publication(J, I).;true;true
2;;advisedby(A, B) :- student(B), student(A).;true;true
2;false;ta(C, A, D), projectmember(E, B).;true;true
2;false,false;ta(F, A, G).;true;true
3;;advisedby(A, B) :- student(A), student(B).;true;true
3;false;ta(C, A, D).;true;true
3;false,true;ta(C, E, F), publication(G, E).;true;true
3;false,true,false;ta(H, I, D), publicati

Results
   AUC ROC   = 0.881084
   AUC PR    = 0.108356
   CLL	      = -0.915122
   Precision = 0.037155 at threshold = 0.5
   Recall    = 1.0
   F1        = 0.071648


Total learning time: 2.652 seconds
Total inference time: 1.556 seconds
AUC ROC: 0.881084


Results scoring model
   AUC ROC   = 0.984375
   AUC PR    = 0.98527
   CLL	      = -0.22548
   Precision = 0.9375 at threshold = 0.5
   Recall    = 0.9375
   F1        = 0.9375


Total scoring time: 0.682 seconds
Refined model CLL: -0.22548


******************************************
Best model found
******************************************
Results
   AUC ROC   = 0.889644
   AUC PR    = 0.153329
   CLL	      = -0.921014
   Precision = 0.036132 at threshold = 0.5
   Recall    = 0.971429
   F1        = 0.069672


Total learning time: 2.941 seconds
Total inference time: 1.099 seconds
AUC ROC: 0.889644
Total revision time: 7.8580000000000005
Best scored revision CLL: -0.213029


[<boostsrl.boostsrl.train object at 0x12525cd00>, {'

In [22]:
import numpy as np
auc_roc = [0.815176, 0.854835, 0.854835]
auc_pr = [0.096136, 0.073558, 0.073558]
cll = [-0.948694,  -0.90546, -0.90546]

print(np.mean(cll), np.std(cll))
print(np.mean(auc_roc), np.std(auc_roc))
print(np.mean(auc_pr), np.std(auc_pr))

-0.9198713333333334 0.02038070305187946
0.8416153333333334 0.018695431890051527
0.081084 0.010643371270419914


## Aleatório

In [26]:
start = time.time()
res_r = genetic(new_src_struct, target, source, train_pos, 
                                         train_neg, train_facts, kb_source, kb_target, pred_target,
                                          NUM_GEN=14, pop_size=30, revision='random')
print("FIM DO GROOT GENETIC: ", time.time()-start)

GENERATION:  0
MELHOR RESULTADO:  -1.0702773072038678
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
BEST: {'m_auc_pr': 0.17426725, 'm_auc_roc': 0.9099392500000001, 'm_cll': -0.563475, 'm_rec': 1.0, 'm_pred': 0.287192625, 'm_f1': 0.137791, 's_auc_pr': 0.052463605505983855, 's_auc_roc': 0.0057985571211724716, 's_cll': 0.04917586191923836, 's_rec': 0.0, 's_prec': 0.21330863588175322, 's_f1': 0.035325067374316496}
GENERATION:  1
MELHOR RESULTADO:  -1.0702773072038678
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
BEST: {'m_auc_pr': 0.17426725, 'm_auc_roc': 0.9099392500000001, 'm_cll': -0.563475, 'm_rec': 1.0, 'm_pred': 0.287192625, 'm_f1': 0.137791, 's_auc_pr': 0.052463605505983855, 's_auc_roc': 0.0057985571211724716, 's_cll': 0.04917586191923836, 's_rec': 0.0, '

In [27]:
res_r

(<src.s_genetic.population.Population at 0x12bf1d790>,
 [-1.0702773072038678,
  -1.0702773072038678,
  -1.0702773072038678,
  -1.074336495445823,
  -1.074336495445823,
  -1.074336495445823,
  -1.074336495445823,
  -1.074336495445823,
  -1.074336495445823,
  -1.074336495445823,
  -1.074336495445823,
  -1.074336495445823],
 [{'m_auc_pr': 0.17426725,
   'm_auc_roc': 0.9099392500000001,
   'm_cll': -0.563475,
   'm_rec': 1.0,
   'm_pred': 0.287192625,
   'm_f1': 0.137791,
   's_auc_pr': 0.052463605505983855,
   's_auc_roc': 0.0057985571211724716,
   's_cll': 0.04917586191923836,
   's_rec': 0.0,
   's_prec': 0.21330863588175322,
   's_f1': 0.035325067374316496},
  {'m_auc_pr': 0.17426725,
   'm_auc_roc': 0.9099392500000001,
   'm_cll': -0.563475,
   'm_rec': 1.0,
   'm_pred': 0.287192625,
   'm_f1': 0.137791,
   's_auc_pr': 0.052463605505983855,
   's_auc_roc': 0.0057985571211724716,
   's_cll': 0.04917586191923836,
   's_rec': 0.0,
   's_prec': 0.21330863588175322,
   's_f1': 0.0353250673

In [28]:
# final_results[f'groot_test:{source}->{target}'] = []
# final_results[f'groot_test_rev:{source}->{target}'] = []
# final_results[f'inf_res:{source}->{target}'] = []
# final_results[f'inf_res_rev:{source}->{target}'] = []

# uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773)
# train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset, 1)
# tr_train_pos = []
# tr_train_neg = []
# tr_train_facts = []

# for i in range(0, len(train_pos)):
#     tr_train_pos.extend(train_pos[i])
#     tr_train_neg.extend(train_neg[i])
#     tr_train_facts.extend(train_facts[i])

# train = [tr_train_pos, tr_train_neg, tr_train_facts]
test = [test_pos, test_neg, test_facts]

individuals = get_k_best_individuals(res_r[0].population, 3)

n_ind = 1
for individual in individuals:
    print("INDIVIDUO ", n_ind)
    refine, transfer = get_refine_transfer(individual, source, target, 'imdb', 'uwcse')
    for i in range(len(train_pos)):
        ttrain = [train_pos[i], train_neg[i][:len(train_pos[i])], train_facts[i]]
        res_ = test_refine_transfer(kb_target, target, refine, transfer, ttrain, test)
        print(res_)
#         final_results[f'groot_test:{source}->{target}'].append(res)
#         import os
    #     thisFile = f'boostsrl/best/test/results_{target.replace('recursion_', '')}.db'
    #     thisFile = f'boostsrl/best/test/results_{target}.db'
    #     base = os.path.splitext(thisFile)[0]
    #     os.rename(thisFile, base + ".txt")
    #     tt = open('boostsrl/best/test/results_advisedby.txt', 'r').readlines()
    #     final = []
    #     for i in tt:
    #         final.append(i.replace('\n', ''))
    #     final_results[f'inf_res:{source}->{target}'].append(final)
        print("===========================================================================")
#         break

    n_ind += 1

INDIVIDUO  1
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.604681, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x12bb12730>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.638636, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x12bc867f0>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.579766, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x12bb120a0>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.525902, 'Precision': [0.044872, 0.5], 'Recall': 1.0, 'F1': 0.08589}, <boostsrl.boostsrl.test object at 0x12bc867f0>)
INDIVIDUO  2
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.604681, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x12bb12790>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.638636, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <bo

## Guided

In [29]:
start = time.time()
res_g = genetic(new_src_struct, target, source, train_pos, 
                                         train_neg, train_facts, kb_source, kb_target, pred_target,
                                          NUM_GEN=14, pop_size=30, revision='guided')
print("FIM DO GROOT GENETIC: ", time.time()-start)

GENERATION:  0
MELHOR RESULTADO:  -1.0675340147162176
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
BEST: {'m_auc_pr': 0.17426725, 'm_auc_roc': 0.9099392500000001, 'm_cll': -0.55824675, 'm_rec': 1.0, 'm_pred': 0.29189149999999997, 'm_f1': 0.15335325, 's_auc_pr': 0.052463605505983855, 's_auc_roc': 0.0057985571211724716, 's_cll': 0.031114475404343582, 's_rec': 0.0, 's_prec': 0.2090936646804489, 's_f1': 0.04759742488504499}
GENERATION:  1
MELHOR RESULTADO:  -1.0675340147162176
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
BEST: {'m_auc_pr': 0.17426725, 'm_auc_roc': 0.9099392500000001, 'm_cll': -0.55824675, 'm_rec': 1.0, 'm_pred': 0.29189149999999997, 'm_f1': 0.15335325, 's_auc_pr': 0.052463605505983855, 's_auc_roc': 0.0057985571211724716, 's_cll': 0.031114475404343582, 's_rec': 

In [30]:
res_g

(<src.s_genetic.population.Population at 0x12b85d1f0>,
 [-1.0675340147162176,
  -1.0675340147162176,
  -1.0675340147162176,
  -1.0675340147162176,
  -1.0675340147162176,
  -1.0675340147162176,
  -1.0675340147162176,
  -1.0675340147162176,
  -1.0675340147162176],
 [{'m_auc_pr': 0.17426725,
   'm_auc_roc': 0.9099392500000001,
   'm_cll': -0.55824675,
   'm_rec': 1.0,
   'm_pred': 0.29189149999999997,
   'm_f1': 0.15335325,
   's_auc_pr': 0.052463605505983855,
   's_auc_roc': 0.0057985571211724716,
   's_cll': 0.031114475404343582,
   's_rec': 0.0,
   's_prec': 0.2090936646804489,
   's_f1': 0.04759742488504499},
  {'m_auc_pr': 0.17426725,
   'm_auc_roc': 0.9099392500000001,
   'm_cll': -0.55824675,
   'm_rec': 1.0,
   'm_pred': 0.29189149999999997,
   'm_f1': 0.15335325,
   's_auc_pr': 0.052463605505983855,
   's_auc_roc': 0.0057985571211724716,
   's_cll': 0.031114475404343582,
   's_rec': 0.0,
   's_prec': 0.2090936646804489,
   's_f1': 0.04759742488504499},
  {'m_auc_pr': 0.17426725,


In [31]:
# final_results[f'groot_test:{source}->{target}'] = []
# final_results[f'groot_test_rev:{source}->{target}'] = []
# final_results[f'inf_res:{source}->{target}'] = []
# final_results[f'inf_res_rev:{source}->{target}'] = []

# uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773)
# train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset, 1)
# tr_train_pos = []
# tr_train_neg = []
# tr_train_facts = []

# for i in range(0, len(train_pos)):
#     tr_train_pos.extend(train_pos[i])
#     tr_train_neg.extend(train_neg[i])
#     tr_train_facts.extend(train_facts[i])

# train = [tr_train_pos, tr_train_neg, tr_train_facts]
test = [test_pos, test_neg, test_facts]

individuals = get_k_best_individuals(res_g[0].population, 3)

n_ind = 1
for individual in individuals:
    print("INDIVIDUO ", n_ind)
    refine, transfer = get_refine_transfer(individual, source, target, 'imdb', 'uwcse')
    for i in range(len(train_pos)):
        ttrain = [train_pos[i], train_neg[i][:len(train_pos[i])], train_facts[i]]
        res_ = test_refine_transfer(kb_target, target, refine, transfer, ttrain, test)
        print(res_)
#         final_results[f'groot_test:{source}->{target}'].append(res)
#         import os
    #     thisFile = f'boostsrl/best/test/results_{target.replace('recursion_', '')}.db'
    #     thisFile = f'boostsrl/best/test/results_{target}.db'
    #     base = os.path.splitext(thisFile)[0]
    #     os.rename(thisFile, base + ".txt")
    #     tt = open('boostsrl/best/test/results_advisedby.txt', 'r').readlines()
    #     final = []
    #     for i in tt:
    #         final.append(i.replace('\n', ''))
    #     final_results[f'inf_res:{source}->{target}'].append(final)
        print("===========================================================================")
#         break

    n_ind += 1

INDIVIDUO  1
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.57913, 'Precision': [0.044872, 0.5], 'Recall': 1.0, 'F1': 0.08589}, <boostsrl.boostsrl.test object at 0x12c52b6a0>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.599783, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x12bb12730>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.563719, 'Precision': [0.044872, 0.5], 'Recall': 1.0, 'F1': 0.08589}, <boostsrl.boostsrl.test object at 0x12c40dc70>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.534539, 'Precision': [0.044872, 0.5], 'Recall': 1.0, 'F1': 0.08589}, <boostsrl.boostsrl.test object at 0x12bb12700>)
INDIVIDUO  2
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.57913, 'Precision': [0.044872, 0.5], 'Recall': 1.0, 'F1': 0.08589}, <boostsrl.boostsrl.test object at 0x12c4b5910>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.599783, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsr

## Mixed

In [23]:
start = time.time()
res_m = genetic(new_src_struct, target, source, train_pos, 
                                         train_neg, train_facts, kb_source, kb_target, pred_target,
                                          NUM_GEN=14, pop_size=30, revision='mixed')
print("FIM DO GROOT GENETIC: ", time.time()-start)

GENERATION:  0
MELHOR RESULTADO:  -1.1094026259325895
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
BEST: {'m_auc_pr': 0.17426725, 'm_auc_roc': 0.9099392500000001, 'm_cll': -0.6346532500000001, 'm_rec': 1.0, 'm_pred': 0.285546, 'm_f1': 0.1320275, 's_auc_pr': 0.052463605505983855, 's_auc_roc': 0.0057985571211724716, 's_cll': 0.054658340691860575, 's_rec': 0.0, 's_prec': 0.21497987868344795, 's_f1': 0.03626637711089984}
GENERATION:  1
MELHOR RESULTADO:  -1.1094026259325895
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
entrei na revisão
BEST: {'m_auc_pr': 0.17426725, 'm_auc_roc': 0.9099392500000001, 'm_cll': -0.6346532500000001, 'm_rec': 1.0, 'm_pred': 0.285546, 'm_f1': 0.1320275, 's_auc_pr': 0.052463605505983855, 's_auc_roc': 0.0057985571211724716, 's_cll': 0.054658340691860575, 's_rec': 0.0, 's

In [24]:
res_m

(<src.s_genetic.population.Population at 0x12b5085e0>,
 [-1.1094026259325895,
  -1.1094026259325895,
  -1.1094026259325895,
  -1.1094026259325895,
  -1.1094026259325895,
  -1.1094026259325895,
  -1.1094026259325895,
  -1.1113716797775453,
  -1.1113716797775453,
  -1.1140241633580599,
  -1.1141969763906763,
  -1.1150832325206157,
  -1.1150832325206157,
  -1.1150832325206157,
  -1.1150832325206157],
 [{'m_auc_pr': 0.17426725,
   'm_auc_roc': 0.9099392500000001,
   'm_cll': -0.6346532500000001,
   'm_rec': 1.0,
   'm_pred': 0.285546,
   'm_f1': 0.1320275,
   's_auc_pr': 0.052463605505983855,
   's_auc_roc': 0.0057985571211724716,
   's_cll': 0.054658340691860575,
   's_rec': 0.0,
   's_prec': 0.21497987868344795,
   's_f1': 0.03626637711089984},
  {'m_auc_pr': 0.17426725,
   'm_auc_roc': 0.9099392500000001,
   'm_cll': -0.6346532500000001,
   'm_rec': 1.0,
   'm_pred': 0.285546,
   'm_f1': 0.1320275,
   's_auc_pr': 0.052463605505983855,
   's_auc_roc': 0.0057985571211724716,
   's_cll': 0

In [25]:
# final_results[f'groot_test:{source}->{target}'] = []
# final_results[f'groot_test_rev:{source}->{target}'] = []
# final_results[f'inf_res:{source}->{target}'] = []
# final_results[f'inf_res_rev:{source}->{target}'] = []

# uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773)
# train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset, 1)
# tr_train_pos = []
# tr_train_neg = []
# tr_train_facts = []

# for i in range(0, len(train_pos)):
#     tr_train_pos.extend(train_pos[i])
#     tr_train_neg.extend(train_neg[i])
#     tr_train_facts.extend(train_facts[i])

# train = [tr_train_pos, tr_train_neg, tr_train_facts]
test = [test_pos, test_neg, test_facts]

individuals = get_k_best_individuals(res_m[0].population, 3)

n_ind = 1
for individual in individuals:
    print("INDIVIDUO ", n_ind)
    refine, transfer = get_refine_transfer(individual, source, target, 'imdb', 'uwcse')
    for i in range(len(train_pos)):
        ttrain = [train_pos[i], train_neg[i][:len(train_pos[i])], train_facts[i]]
        res_ = test_refine_transfer(kb_target, target, refine, transfer, ttrain, test)
        print(res_)
#         final_results[f'groot_test:{source}->{target}'].append(res)
#         import os
    #     thisFile = f'boostsrl/best/test/results_{target.replace('recursion_', '')}.db'
    #     thisFile = f'boostsrl/best/test/results_{target}.db'
    #     base = os.path.splitext(thisFile)[0]
    #     os.rename(thisFile, base + ".txt")
    #     tt = open('boostsrl/best/test/results_advisedby.txt', 'r').readlines()
    #     final = []
    #     for i in tt:
    #         final.append(i.replace('\n', ''))
    #     final_results[f'inf_res:{source}->{target}'].append(final)
        print("===========================================================================")
#         break

    n_ind += 1

INDIVIDUO  1
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.6682, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x12b4681c0>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.714954, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x12bc867f0>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.655864, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x12bc86c40>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.604685, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x12b4681c0>)
INDIVIDUO  2
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.6682, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boostsrl.boostsrl.test object at 0x12bc86c40>)
({'AUC ROC': 0.90215, 'AUC PR': 0.089744, 'CLL': -0.714954, 'Precision': [0.036881, 0.5], 'Recall': 1.0, 'F1': 0.071138}, <boost

## Teste TreeBoostler

In [9]:
tr_train_pos = []
tr_train_neg = []
tr_train_facts = []

for i in range(0, len(train_pos)):
    tr_train_pos.extend(train_pos[i])
    tr_train_neg.extend(train_neg[i])
    tr_train_facts.extend(train_facts[i])
test = [test_pos, test_neg, test_facts]
aa = []
for i in range(0, len(train_pos)):
    train = [train_pos[i], train_neg[i][:2*len(train_pos[i])], train_facts[i]]
    aa.append(test_tree_b(source, target, kb_source, kb_target, structured_src, train, test))
# aa = test_tree_b(source, target, kb_source, kb_target, structured_src, train, test)

******************************************
Performing Parameter Learning
******************************************
Refine
['workedunder(A, B)', {'': 'director(B), actor(A)', 'true': 'movie(C, A), movie(C, B)'}, {'true,true': [1.43e-06, 0, 382], 'true,false': [6.99e-08, 71, 0], 'false': [0.0, 728, 0]}]
['workedunder(A, B)', {'': 'director(B), actor(A)', 'true': 'movie(C, A), movie(C, B)'}, {'true,true': [0.0, 0, 382], 'true,false': [0.0, 54, 0], 'false': [0.0, 724, 0]}]
['workedunder(A, B)', {'': 'director(B), actor(A)', 'true': 'movie(C, A), movie(C, B)'}, {'true,true': [0.0, 0, 382], 'true,false': [4.08e-08, 57, 0], 'false': [0.0, 761, 0]}]
['workedunder(A, B)', {'': 'actor(A), director(B)', 'true': 'movie(C, A), movie(C, B)'}, {'true,true': [0.0, 0, 382], 'true,false': [4.47e-08, 61, 0], 'false': [4.48e-07, 719, 0]}]
['workedunder(A, B)', {'': 'director(B), actor(A)', 'true': 'movie(C, A), movie(C, B)'}, {'true,true': [0.0, 0, 382], 'true,false': [0.0, 61, 0], 'false': [0.0, 709, 0]

WILL Produced-Tree #1
% FOR advisedby(A, B):
%   if ( professor(B), student(A) )
%   then if ( publication(C, A), publication(C, B) )
%   | then return 0.8581489350995123;  // std dev = 0,000, 9,000 (wgt'ed) examples reached here.  /* #pos=9 */
%   | else if ( ta(D, A, E), publication(F, B) )
%   | | then if ( publication(G, A), tempadvisedby(H, B) )
%   | | | then return 0.3581489350995122;  // std dev = 1,000, 4,000 (wgt'ed) examples reached here.  /* #neg=2 #pos=2 */
%   | | | else return 0.8581489350995122;  // std dev = 0,000, 5,000 (wgt'ed) examples reached here.  /* #pos=5 */
%   | | else if ( tempadvisedby(I, B), ta(J, I, K) )
%   | | | then return 0.6081489350995122;  // std dev = 0,866, 4,000 (wgt'ed) examples reached here.  /* #neg=1 #pos=3 */
%   | | | else return 0.0581489350995122;  // std dev = 0,894, 5,000 (wgt'ed) examples reached here.  /* #neg=4 #pos=1 */
%   else return -0.1418510649004877;  // std dev = 3,16e-08, 33,000 (wgt'ed) examples reached here.  /* #neg=33 *

Results
   AUC ROC   = 0.93284
   AUC PR    = 0.27654
   CLL	      = -0.187612
   Precision = 0.140625 at threshold = 0.5
   Recall    = 0.514286
   F1        = 0.220859


Total learning time: 3.626 seconds
Total inference time: 1.279 seconds
AUC ROC: 0.93284


Results scoring model
   AUC ROC   = 0.996875
   AUC PR    = 0.994096
   CLL	      = -0.179335
   Precision = 0.95 at threshold = 0.5
   Recall    = 0.95
   F1        = 0.95


Total scoring time: 0.526 seconds
Refined model CLL: -0.179335


Refining iteration 2
********************************
Candidate for revision
0;;advisedby(A, B) :- professor(B), student(A).;true;true
0;true;publication(C, A), publication(C, B).;true;true
0;true,false;ta(D, A, E), publication(F, B).;true;true
0;true,false,true;publication(G, A), tempadvisedby(H, B).;true;true
1;;advisedby(A, B) :- professor(B), student(A).;true;true
1;true;publication(C, A), publication(C, B).;true;true
1;true,false;publication(D, B), ta(E, A, F).;true;true
1;true,false,tru

Results
   AUC ROC   = 0.93284
   AUC PR    = 0.27654
   CLL	      = -0.187612
   Precision = 0.140625 at threshold = 0.5
   Recall    = 0.514286
   F1        = 0.220859


Total learning time: 3.69 seconds
Total inference time: 1.406 seconds
AUC ROC: 0.93284


Results scoring model
   AUC ROC   = 0.996875
   AUC PR    = 0.994096
   CLL	      = -0.179335
   Precision = 0.95 at threshold = 0.5
   Recall    = 0.95
   F1        = 0.95


Total scoring time: 0.536 seconds
Refined model CLL: -0.179335


******************************************
Best model found
******************************************
Results
   AUC ROC   = 0.93284
   AUC PR    = 0.27654
   CLL	      = -0.187612
   Precision = 0.140625 at threshold = 0.5
   Recall    = 0.514286
   F1        = 0.220859


Total learning time: 3.626 seconds
Total inference time: 1.279 seconds
AUC ROC: 0.93284
Total revision time: 9.684
Best scored revision CLL: -0.179335


******************************************
Performing Parameter Learni

Results
   AUC ROC   = 0.90215
   AUC PR    = 0.089744
   CLL	      = -0.325935
   Precision = 0.089744 at threshold = 0.5
   Recall    = 1.0
   F1        = 0.164706


Total learning time: 0.792 seconds
Total inference time: 0.861 seconds
AUC ROC: 0.90215


Results scoring model
   AUC ROC   = 0.944444
   AUC PR    = 0.818182
   CLL	      = -0.234561
   Precision = 0.818182 at threshold = 0.5
   Recall    = 1.0
   F1        = 0.9


Total scoring time: 0.408 seconds
Parameter learned model CLL: -0.234561


Structure after Parameter Learning
[['advisedby(A, B)', {'': 'professor(B), student(A)', 'true': 'yearsinprogram(A, C), yearsinprogram(B, C)'}, {'true,true': [0.0, 0, 0], 'true,false': [1.279, 2, 9], 'false': [0.0, 16, 0]}], ['advisedby(A, B)', {'': 'professor(B), student(A)', 'true': 'yearsinprogram(A, C), yearsinprogram(B, C)'}, {'true,true': [0.0, 0, 0], 'true,false': [1.279, 2, 9], 'false': [7.45e-09, 16, 0]}], ['advisedby(A, B)', {'': 'professor(B), student(A)', 'true': 'yearsinp

...

Results
   AUC ROC   = 0.887092
   AUC PR    = 0.070606
   CLL	      = -0.401157
   Precision = 0.098802 at threshold = 0.5
   Recall    = 0.942857
   F1        = 0.178862


Total learning time: 3.475 seconds
Total inference time: 1.14 seconds
AUC ROC: 0.887092


Results scoring model
   AUC ROC   = 0.988522
   AUC PR    = 0.9762
   CLL	      = -0.191053
   Precision = 0.935484 at threshold = 0.5
   Recall    = 0.878788
   F1        = 0.90625


Total scoring time: 0.665 seconds
Refined model CLL: -0.191053


******************************************
Best model found
******************************************
Results
   AUC ROC   = 0.887069
   AUC PR    = 0.070593
   CLL	      = -0.400522
   Precision = 0.098802 at threshold = 0.5
   Recall    = 0.942857
   F1        = 0.178862


Total learning time: 4.722 seconds
Total inference time: 1.205 seconds
AUC ROC: 0.887069
Total revision time: 15.635000000000002
Best scored revision CLL: -0.183611


******************************************

Results
   AUC ROC   = 0.937707
   AUC PR    = 0.29769
   CLL	      = -0.225598
   Precision = 0.089744 at threshold = 0.5
   Recall    = 1.0
   F1        = 0.164706


Total learning time: 1.205 seconds
Total inference time: 0.893 seconds
AUC ROC: 0.937707


Results scoring model
   AUC ROC   = 0.90625
   AUC PR    = 0.808336
   CLL	      = -0.323212
   Precision = 0.666667 at threshold = 0.5
   Recall    = 1.0
   F1        = 0.8


Total scoring time: 0.427 seconds
Parameter learned model CLL: -0.323212


Structure after Parameter Learning
[['advisedby(A, B)', {'': 'professor(B), student(A)', 'true': 'publication(C, A), publication(C, B)'}, {'true,true': [0.0, 0, 4], 'true,false': [2.191, 8, 12], 'false': [1.49e-08, 24, 0]}], ['advisedby(A, B)', {'': 'professor(B), student(A)', 'true': 'publication(C, A), publication(C, B)'}, {'true,true': [0.0, 0, 4], 'true,false': [2.191, 8, 12], 'false': [0.0, 24, 0]}], ['advisedby(A, B)', {'': 'professor(B), student(A)', 'true': 'publication(C, A),

Results
   AUC ROC   = 0.940613
   AUC PR    = 0.299028
   CLL	      = -0.193231
   Precision = 0.2 at threshold = 0.5
   Recall    = 0.542857
   F1        = 0.292308


Total learning time: 5.123 seconds
Total inference time: 1.226 seconds
AUC ROC: 0.940613


Results scoring model
   AUC ROC   = 0.980469
   AUC PR    = 0.966942
   CLL	      = -0.236052
   Precision = 1.0 at threshold = 0.5
   Recall    = 0.8125
   F1        = 0.896552


Total scoring time: 0.637 seconds
Refined model CLL: -0.236052


Refining iteration 2
********************************
Candidate for revision
0;;advisedby(A, B) :- professor(B), student(A).;true;true
0;true;publication(C, A), publication(C, B).;true;true
0;true,false;ta(D, A, E), projectmember(F, B).;true;true
0;true,false,false;projectmember(G, B).;true;true
1;;advisedby(A, B) :- professor(B), student(A).;true;true
1;true;publication(C, A), publication(C, B).;true;true
1;true,false;ta(D, A, E), publication(F, B).;true;true
1;true,false,true;ta(G, H, E)

Results
   AUC ROC   = 0.937533
   AUC PR    = 0.295876
   CLL	      = -0.195675
   Precision = 0.2 at threshold = 0.5
   Recall    = 0.542857
   F1        = 0.292308


Total learning time: 4.959 seconds
Total inference time: 1.61 seconds
AUC ROC: 0.937533


Results scoring model
   AUC ROC   = 0.980469
   AUC PR    = 0.966942
   CLL	      = -0.234372
   Precision = 1.0 at threshold = 0.5
   Recall    = 0.8125
   F1        = 0.896552


Total scoring time: 0.668 seconds
Refined model CLL: -0.234372


Refining iteration 3
********************************
Candidate for revision
0;;advisedby(A, B) :- professor(B), student(A).;true;true
0;true;publication(C, A), publication(C, B).;true;true
0;true,false;ta(D, A, E), projectmember(F, B).;true;true
0;true,false,false;projectmember(G, B).;true;true
1;;advisedby(A, B) :- professor(B), student(A).;true;true
1;true;publication(C, A), publication(C, B).;true;true
1;true,false;ta(D, A, E), publication(F, B).;true;true
1;true,false,true;ta(G, H, E),

Results
   AUC ROC   = 0.940613
   AUC PR    = 0.299028
   CLL	      = -0.193231
   Precision = 0.2 at threshold = 0.5
   Recall    = 0.542857
   F1        = 0.292308


Total learning time: 5.574 seconds
Total inference time: 1.213 seconds
AUC ROC: 0.940613


Results scoring model
   AUC ROC   = 0.980469
   AUC PR    = 0.966942
   CLL	      = -0.236052
   Precision = 1.0 at threshold = 0.5
   Recall    = 0.8125
   F1        = 0.896552


Total scoring time: 0.735 seconds
Refined model CLL: -0.236052


******************************************
Best model found
******************************************
Results
   AUC ROC   = 0.937533
   AUC PR    = 0.295876
   CLL	      = -0.195675
   Precision = 0.2 at threshold = 0.5
   Recall    = 0.542857
   F1        = 0.292308


Total learning time: 4.959 seconds
Total inference time: 1.61 seconds
AUC ROC: 0.937533
Total revision time: 19.328
Best scored revision CLL: -0.234372




In [23]:
aa[3]

 {'AUC ROC': 0.937533,
  'AUC PR': 0.295876,
  'CLL': -0.195675,
  'Precision': [0.2, 0.5],
  'Recall': 0.542857,
  'F1': 0.292308,
  'Learning time': 4.959,
  'Inference time': 1.61},
 [['advisedby(A, B)',
   {'': 'professor(B), student(A)',
    'true': 'publication(C, A), publication(C, B)',
    'true,false': 'ta(D, A, E), projectmember(F, B)',
    'true,false,false': 'projectmember(G, B)'},
   {'true,true': [0.0, 0, 4],
    'true,false,true': [0.816, 2, 1],
    'true,false,false,true': [2.11e-08, 0, 3],
    'true,false,false,false': [1.852, 6, 8],
    'false': [1.49e-08, 24, 0]}],
  ['advisedby(A, B)',
   {'': 'professor(B), student(A)',
    'true': 'publication(C, A), publication(C, B)',
    'true,false': 'ta(D, A, E), publication(F, B)',
    'true,false,true': 'ta(G, H, E), publication(I, H)',
    'true,false,false': 'projectmember(J, B)'},
   {'true,true': [0.0, 0, 4],
    'true,false,true,true': [0.0, 0, 4],
    'true,false,true,false': [1.415, 4, 4],
    'true,false,false,true'

In [13]:
background_knowledge_src = boostsrl.modes(kb_target, [target], useStdLogicVariables=False, 
                                      maxTreeDepth=3, nodeSize=2, numOfClauses=8)
sys.path.insert(0, '../TreeBoostler')

from revision import revision

def print_function(message):
        global experiment_title
        global nbr
        experiment_title = f'Twitter->Yeast2: {source}->{target}'
        nbr = 1
        if not os.path.exists('experiments/' + experiment_title):
            os.makedirs('experiments/' + experiment_title)
        with open('experiments/' + experiment_title + '/' + str(nbr) + '_' + experiment_title + '.txt', 'a') as f:
            print(message, file=f)
            print(message)
model = []
for i in range(0, len(train_pos)):
#     model = revision.learn_test_model(background_knowledge_src, boostsrl, target, tr_train_pos, tr_train_neg, tr_train_facts, test_pos, test_neg, test_facts, trees=10, print_function=print_function)
    model.append(revision.learn_test_model(background_knowledge_src, boostsrl, target, train_pos[i], train_neg[i][:2*len(train_pos[i])], train_facts[i], test_pos, test_neg, test_facts, trees=10, print_function=print_function))

WILL Produced-Tree #1
% FOR advisedby(A, B):
%   if ( professor(B), publication(C, B) )
%   then if ( publication(C, A) )
%   | then return 0.8581489350995123;  // std dev = 0,000, 9,000 (wgt'ed) examples reached here.  /* #pos=9 */
%   | else if ( ta(D, A, E) )
%   | | then if ( publication(F, A), tempadvisedby(G, B) )
%   | | | then return 0.3581489350995122;  // std dev = 1,000, 4,000 (wgt'ed) examples reached here.  /* #neg=2 #pos=2 */
%   | | | else return 0.8581489350995122;  // std dev = 0,000, 5,000 (wgt'ed) examples reached here.  /* #pos=5 */
%   | | else return 0.35814893509951223;  // std dev = 1,225, 6,000 (wgt'ed) examples reached here.  /* #neg=3 #pos=3 */
%   else if ( student(B) )
%   | then return -0.1418510649004877;  // std dev = 3,16e-08, 33,000 (wgt'ed) examples reached here.  /* #neg=33 */
%   | else return 0.19148226843284552;  // std dev = 0,816, 3,000 (wgt'ed) examples reached here.  /* #neg=2 #pos=1 */
WILL Produced-Tree #2
% FOR advisedby(A, B):
%   if ( pro

Results
   AUC ROC   = 0.889471
   AUC PR    = 0.170115
   CLL	      = -0.260154
   Precision = 0.098901 at threshold = 0.5
   Recall    = 0.514286
   F1        = 0.165899


Total learning time: 4.906 seconds
Total inference time: 1.422 seconds
AUC ROC: 0.889471


WILL Produced-Tree #1
% FOR advisedby(A, B):
%   if ( professor(B) )
%   then if ( tempadvisedby(C, B), ta(D, A, E) )
%   | then return 0.8581489350995121;  // std dev = 2,11e-08, 3,000 (wgt'ed) examples reached here.  /* #pos=3 */
%   | else return 0.6081489350995122;  // std dev = 0,433, 8,000 (wgt'ed) examples reached here.  /* #neg=2 #pos=6 */
%   else return -0.1418510649004878;  // std dev = 0,000, 16,000 (wgt'ed) examples reached here.  /* #neg=16 */
WILL Produced-Tree #2
% FOR advisedby(A, B):
%   if ( professor(B) )
%   then if ( tempadvisedby(C, B), ta(D, A, E) )
%   | then return 0.7194734122109545;  // std dev = 0,000, 3,000 (wgt'ed) examples reached here.  /* #pos=3 */
%   | else return 0.5170719622367217;  // st

Results
   AUC ROC   = 0.885462
   AUC PR    = 0.15505
   CLL	      = -0.593596
   Precision = 0.065606 at threshold = 0.5
   Recall    = 0.942857
   F1        = 0.122677


Total learning time: 7.779 seconds
Total inference time: 1.233 seconds
AUC ROC: 0.885462


WILL Produced-Tree #1
% FOR advisedby(A, B):
%   if ( professor(B) )
%   then if ( publication(C, A) )
%   | then return 0.8581489350995122;  // std dev = 0,000, 4,000 (wgt'ed) examples reached here.  /* #pos=4 */
%   | else if ( ta(D, A, E), projectmember(F, B) )
%   | | then return 0.19148226843284552;  // std dev = 0,816, 3,000 (wgt'ed) examples reached here.  /* #neg=2 #pos=1 */
%   | | else if ( projectmember(G, B) )
%   | | | then return 0.8581489350995121;  // std dev = 2,11e-08, 3,000 (wgt'ed) examples reached here.  /* #pos=3 */
%   | | | else return 0.42957750652808374;  // std dev = 1,852, 14,000 (wgt'ed) examples reached here.  /* #neg=6 #pos=8 */
%   else return -0.14185106490048774;  // std dev = 1,49e-08, 24,000

Results
   AUC ROC   = 0.855796
   AUC PR    = 0.07068
   CLL	      = -0.445039
   Precision = 0.062678 at threshold = 0.5
   Recall    = 0.628571
   F1        = 0.11399


Total learning time: 5.42 seconds
Total inference time: 1.061 seconds
AUC ROC: 0.855796




In [26]:
model[0]

 {'AUC ROC': 0.889471,
  'AUC PR': 0.170115,
  'CLL': -0.260154,
  'Precision': [0.098901, 0.5],
  'Recall': 0.514286,
  'F1': 0.165899,
  'Learning time': 4.906,
  'Inference time': 1.422},
 [['advisedby(A, B)',
   {'': 'professor(B), publication(C, B)',
    'true': 'publication(C, A)',
    'true,false': 'ta(D, A, E)',
    'true,false,true': 'publication(F, A), tempadvisedby(G, B)',
    'false': 'student(B)'},
   {'true,true': [0.0, 0, 9],
    'true,false,true,true': [1.0, 2, 2],
    'true,false,true,false': [0.0, 0, 5],
    'true,false,false': [1.225, 3, 3],
    'false,true': [3.16e-08, 33, 0],
    'false,false': [0.816, 2, 1]}],
  ['advisedby(A, B)',
   {'': 'professor(B), publication(C, B)',
    'true': 'publication(C, A)',
    'true,false': 'ta(D, A, E)',
    'true,false,true': 'tempadvisedby(F, B), ta(G, F, H)',
    'false': 'student(B)'},
   {'true,true': [2.98e-08, 0, 9],
    'true,false,true,true': [0.911, 2, 2],
    'true,false,true,false': [0.098, 0, 5],
    'true,false,fals

In [29]:
model[3]

 {'AUC ROC': 0.855796,
  'AUC PR': 0.07068,
  'CLL': -0.445039,
  'Precision': [0.062678, 0.5],
  'Recall': 0.628571,
  'F1': 0.11399,
  'Learning time': 5.42,
  'Inference time': 1.061},
 [['advisedby(A, B)',
   {'': 'professor(B)',
    'true': 'publication(C, A)',
    'true,false': 'ta(D, A, E), projectmember(F, B)',
    'true,false,false': 'projectmember(G, B)'},
   {'true,true': [0.0, 0, 4],
    'true,false,true': [0.816, 2, 1],
    'true,false,false,true': [2.11e-08, 0, 3],
    'true,false,false,false': [1.852, 6, 8],
    'false': [1.49e-08, 24, 0]}],
  ['advisedby(A, B)',
   {'': 'professor(B)',
    'true': 'publication(C, A)',
    'true,false': 'ta(D, A, E), publication(F, B)',
    'true,false,true': 'ta(G, H, E), publication(I, H)',
    'true,false,false': 'projectmember(J, B)'},
   {'true,true': [0.0, 0, 4],
    'true,false,true,true': [0.0, 0, 4],
    'true,false,true,false': [1.415, 4, 4],
    'true,false,false,true': [0.768, 1, 3],
    'true,false,false,false': [0.866, 3, 1